In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

def detect_lanes(frame):
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply Canny edge detection
    edges = cv2.Canny(blur, 50, 150)

    # Define region of interest
    height, width = frame.shape[:2]
    vertices = np.array([[(0, height), (width/2, height/2), (width, height)]], dtype=np.int32)
    roi = region_of_interest(edges, vertices)

    # Detect lines using Hough Transform
    lines = cv2.HoughLinesP(roi, 2, np.pi/180, 100, minLineLength=40, maxLineGap=5)

    # Draw lines on the frame
    line_img = np.zeros((height, width, 3), dtype=np.uint8)
    if lines is not None:
        draw_lines(line_img, lines)

    # Overlay lines on the original frame
    result = cv2.addWeighted(frame, 0.8, line_img, 1, 0)

    return result

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, vertices, 255)
    masked_img = cv2.bitwise_and(img, mask)
    return masked_img

def draw_lines(img, lines, color=[255, 0, 0], thickness=3):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

# Read video file
cap = cv2.VideoCapture('/content/sample_data/test_video.mp4')

# Get video details
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define code and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video.mp4', fourcc, fps, (frame_width, frame_height))

while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame for lane detection
    result_frame = detect_lanes(frame)

    # Write the resulting frame to the output video file
    out.write(result_frame)

    # Display the resulting frame
    cv2_imshow(result_frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and writer, and close all windows
cap.release()
out.release()
cv2.destroyAllWindows()